# tf.function()的功能和作用
其使用方法是为了代替原来的session()的。

## 回顾session的用法
Tensorflow的逻辑和一般的编程语言不同的在于，TensorFlow是先建立graph（计算的结构），并不计算。而后选择好input和output，把graph放在session里面去运行。

这里graph指的是：与alghrithms的图相同，即为运算节点与数据流一同构成的。因此当你建立好graph之后，你就已经建立好了所有计算规则和规范。

而session的作用是给graph提供一个运行的环境，通过session.run()，input在session中被加载到graph中，而graph的output在session被计算出，并且session可以记录一些被追踪量，从而对被追踪量进行更新。

因此session的计算流程如下：
1. 初始化一个计算图并且将该计算图设置为当前scope下的默认计算图
2. 用TF API设计计算图(比如: y=tf.matmul(a, x) + b)
3. 提前界定好参数共享并划分相应的参数scope
4. 创建并配置好tf.Session
5. 将计算图传给tf.Session
6. 初始化参数

用tf.Session.run来执行计算图的节点, 被执行的节点会反向追踪所有依赖的需要执行的节点并执行计算.



### 1X中的例子

In [8]:
import tensorflow.compat.v1 as tf1


g = tf1.Graph() #初始化计算图
with g.as_default(): # 设置为默认计算图
    a = tf1.constant([[10,10],[11.,1.]]) 
    x = tf1.constant([[1.,0.],[0.,1.]])
    b = tf1.Variable(12.)
    y = tf1.matmul(a, x) + b # 描述计算图
    init_op = tf1.global_variables_initializer() # 待执行节点

#先建立好一个graph，一个数据计算结构，然而他什么也没有做，没有输入，没有输出，只是一个单独的结构而已。



In [12]:
#开始运行，给input， 放到graph（既定的数据计算结构），得到输出y。

with tf1.Session() as sess: # 配置会话
    sess.run(init_op) # 执行节点
    print(sess.run(y)) # 输出结果


RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

### eage excecution
逻辑变成通常的计算逻辑，给出参数值，给出变量，给出模型，给出input，得到计算的值。

In [10]:
import tensorflow as tf
#定义一些参数和变量
a = tf.constant([[10,10],[11.,1.]])
x = tf.constant([[1.,0.],[0.,1.]])
b = tf.Variable(12.)

#直接计算
y = tf.matmul(a, x) + b
print(y.numpy())


[[22. 22.]
 [23. 13.]]


## 优缺点
既然eage execuation这么好，为什么还要用session？

因为静态图的计算效率要远大于动态计算的效率！！  第一种方法就是静态图，而第二种方法是动态计算。 而这种计算效率的差距会随着计算的复杂性的增加而被放大。


## tf.function( )的提出
function的提出就是为了解决这个问题的！

function()函数一方面去掉了奇怪的逻辑（spaceholder, session 等）。另一方面，可以用很自然的逻辑来建立静态图，而后运行。

__我们试图想直接把eage execution中执行的内容，原封不动的，直接封装在function之下。__

In [13]:
@tf.function
def f():
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.)
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    tf.print("TF-PRINT: ", y)
    return y


这样做就会报错，因为，动态图和静态图的作用域不同，variable是一个持续的节点，不受python作用域的影响，但在动态图中，这个variable在使用过后就被销毁了

In [14]:
f()

PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)


ValueError: in converted code:

    <ipython-input-13-4a4cf75eb2a6>:5 f  *
        b = tf.Variable(12.)
    /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:262 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:60 getter
        return captured_getter(captured_previous, **kwargs)
    /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:364 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


为了让variable在静态图和动态图可以相互兼容，我们只需要将variable，定义为动态变量，并变成传入参数，传入到静态图中，即可。

In [17]:
@tf.function
def f(b):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    tf.print("TF-PRINT: ", y)
    return y

#我们把变量作为传入参数来使用即可。
b = tf.Variable(12.)
f(b)


PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[22 22]
 [23 13]]


<tf.Tensor: id=175, shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

__也可以把变量变成类属性来调用__\
这里我们把variable变成init的部分，调用的时候，通过实例化对variable进行赋值。

In [40]:
class F():
    def __init__(self,b = None):
        self._b = b

    @tf.function
    def __call__(self):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        if self._b is None:
            self._b = tf.Variable(12.)
        y = tf.matmul(a, x) + self._b
        print("PRINT: ", y)
        tf.print("TF-PRINT: ", y)
        return y



In [41]:
f = F(tf.Variable(4.))
f()

PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[14 14]
 [15 5]]


<tf.Tensor: id=484, shape=(2, 2), dtype=float32, numpy=
array([[14., 14.],
       [15.,  5.]], dtype=float32)>